In [ ]:
--1. Listar para cada track el nombre del mismo, el genero y el media type

In [ ]:
--2. Listar la cantidad de tracks que tiene cada genero.
SELECT g.Name AS genero, COUNT(T.TrackId) AS cantidad
FROM Chinook.dbo.Genre AS g
INNER JOIN Chinook.dbo.Track t ON t.GenreId = g.GenreId
GROUP BY g.Name
ORDER BY cantidad ASC;

In [ ]:
--Obtener los artistas que no tienen albumes. Dar al menos dos soluciones distintas.
SELECT a.Name AS artista
FROM Chinook.dbo.Artist AS a
LEFT OUTER JOIN Chinook.dbo.Album album ON a.ArtistId = album.ArtistId
WHERE album.AlbumId IS NULL;

SELECT a.Name AS artista
FROM Chinook.dbo.Artist AS a
WHERE a.ArtistId NOT IN (
    SELECT DISTINCT Album.ArtistId FROM Chinook.dbo.Album
);

In [83]:
-- Listar el nombre y la cantidad de tracks de los artistas con mas de 50 tracks, ordenado por cantidad de tracks de forma ascendente.

WITH artist_album AS (
    SELECT artist.ArtistId, artist.Name AS artist, album.AlbumId
    FROM Chinook.dbo.Artist artist
    INNER JOIN Chinook.dbo.Album album ON album.ArtistId = artist.ArtistId
)

SELECT artist_album.artist, COUNT(track.TrackId) AS tracks_quantity
FROM artist_album
INNER JOIN Chinook.dbo.Track track ON artist_album.AlbumId = track.AlbumId
GROUP BY artist_album.artist
HAVING COUNT(track.TrackId) > 50
ORDER BY tracks_quantity ASC;

(12 rows affected)

Total execution time: 00:00:00.017

artist,tracks_quantity
Faith No More,52
Van Halen,52
The Office,53
Various Artists,56
Lenny Kravitz,57
Pearl Jam,67
Lost,92
Deep Purple,92
Metallica,112
Led Zeppelin,114


In [86]:
-- Para cada cliente obtener la cantidad de empleados que viven en la misma ciudad ordenados descendentemente por cantidad de empleados.

SELECT *
FROM Chinook.dbo.Customer customer, Chinook.dbo.Employee employee
WHERE  customer.City = employee.City;

(1 row affected)

Total execution time: 00:00:00.013

CustomerId,FirstName,LastName,Company,Address,City,State,Country,PostalCode,Phone,Fax,Email,SupportRepId,EmployeeId,LastName,FirstName,Title,ReportsTo,BirthDate,HireDate,Address,City,State,Country,PostalCode,Phone,Fax,Email
14,Mark,Philips,Telus,8210 111 ST NW,Edmonton,AB,Canada,T6G 2C7,+1 (780) 434-4554,+1 (780) 434-5565,mphilips12@shaw.ca,5,1,Adams,Andrew,General Manager,NULL,1962-02-18 00:00:00.000,2002-08-14 00:00:00.000,11120 Jasper Ave NW,Edmonton,AB,Canada,T5K 2N1,+1 (780) 428-9482,+1 (780) 428-3457,andrew@chinookcorp.com


In [15]:
-- Obtener el dinero recaudado por cada empleado durante cada anio. ¿Como extraer un campo de una fecha?
 

WITH invoice_by_employee AS (
    SELECT InvoiceDate, customer.CustomerId, Total, SupportRepId AS EmployeeId
    FROM Chinook.dbo.Invoice invoice
    INNER JOIN Chinook.dbo.Customer customer ON customer.CustomerId = invoice.CustomerId
    INNER JOIN Chinook.dbo.Employee employee ON employee.EmployeeId = customer.SupportRepId
)
-- SELECT * FROM invoice_by_employee;
SELECT EmployeeId, SUM(Total) AS Tarasca, YEAR(InvoiceDate) AS 'Year'
FROM invoice_by_employee
GROUP BY YEAR(InvoiceDate), EmployeeId;

(15 rows affected)

Total execution time: 00:00:00.205

EmployeeId,Tarasca,Year
3,123.75,2009
3,221.92,2010
3,184.34,2011
3,146.60,2012
3,156.43,2013
4,161.37,2009
4,122.76,2010
4,125.77,2011
4,197.20,2012
4,168.30,2013


In [50]:
-- Obtener la cantidad de pistas de audio que tengan una duracion superior a la duracion promedio de todas las pistas de audio. 
-- Ademas, obtener la sumatoria de la duracion de todas esas pistas en minutos.


WITH mayores_a_la_media AS (
    SELECT Track.TrackId, Track.Milliseconds FROM Chinook.dbo.Track
    INNER JOIN Chinook.dbo.MediaType ON MediaType.MediaTypeId = Track.MediaTypeId
    WHERE 
        Milliseconds > (SELECT AVG(Milliseconds) FROM Chinook.dbo.Track)
        AND MediaType.Name LIKE '%audio%' 
)
SELECT COUNT(*) AS Total, (SUM(Milliseconds) / 60000) AS DuracionTotal 
FROM mayores_a_la_media;

(1 row affected)

Total execution time: 00:00:00.148

Total,DuracionTotal
282,2453


In [60]:
-- 8. (a) Crear una vista que devuelva las playlists que tienen al menos una
-- pista del genero “Rock”.


-- IF OBJECT_ID('RockyPlaylists', 'V') IS NOT NULL
-- DROP VIEW RockyPlaylists
-- GO

CREATE VIEW RockPlaylists AS (
    SELECT Playlist.PlaylistId, Playlist.Name AS PlaylistName 
    FROM Chinook.dbo.Playlist
    WHERE PlaylistId  IN (
        SELECT PlaylistTrack.PlaylistId 
        FROM Chinook.dbo.PlaylistTrack
        INNER JOIN Chinook.dbo.Track ON Track.TrackId = PlaylistTrack.TrackId
        INNER JOIN Chinook.dbo.Genre ON Track.GenreId = Genre.GenreId
        WHERE Genre.Name = 'Rock'
    )
);

: Msg 2714, Level 16, State 3, Procedure RockPlaylists, Line 9
There is already an object named 'RockPlaylists' in the database.

Total execution time: 00:00:00.121

In [65]:
-- (b) Obtener de forma concisa la cantidad de playlists que no poseen
-- pistas de dicho genero.

SELECT COUNT(*) AS NonRockPlaylists
FROM Chinook.dbo.Playlist
WHERE Playlist.PlaylistId NOT IN (
    SELECT RockPlaylists.PlaylistId 
    FROM RockPlaylists
);


(1 row affected)

Total execution time: 00:00:00.187

NonRockPlaylists
13


In [74]:
-- 9. Obtener las playlists mas caras. (Ayuda: primero obtener el ‘precio’
-- de cada playlist.)
WITH PrecioPorPlaylist AS (
    SELECT Playlist.PlaylistId, SUM(Track.UnitPrice) AS Precio FROM Chinook.dbo.PlaylistTrack
    INNER JOIN Chinook.dbo.Playlist ON Playlist.PlaylistId = PlaylistTrack.PlaylistId
    INNER JOIN Chinook.dbo.Track ON Track.TrackId = PlaylistTrack.TrackId
    GROUP BY Playlist.PlaylistId
)
SELECT *
FROM PrecioPorPlaylist
WHERE PrecioPorPlaylist.Precio = (
    SELECT MAX(PrecioPorPlaylist.Precio) FROM PrecioPorPlaylist
)

(2 rows affected)

Total execution time: 00:00:00.165

PlaylistId,Precio
1,3257.10
8,3257.10
